In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ust_wildfire.temperature import average_yearly_temp, average_monthly_temp
from ust_wildfire.wildfire_stat import filter_lifetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


plt.rcParams.update({
    "figure.dpi": 100,
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"]})
plt.rc("text.latex", preamble=r"\usepackage{newpxmath}")

In [ ]:
# Load the data
df_fire = pd.read_csv(f"../data/alberta.csv")
df_ystat = pd.read_csv(f"../data/results/alberta_yearly_stat.csv")
df_mstat = pd.read_csv(f"../data/resuluts/alberta_monthly_stat.csv")
df_temp = pd.read_csv(f"../data/alberta_monthly_temp.csv")

# Get the data in the time range of interest
years = np.arange(1961, 2019)
temp = average_yearly_temp(df_temp, years)
temp_m = average_monthly_temp(df_temp)
df_ystat = df_ystat[(df_ystat["year"].isin(years))]


# Exclude events with undefined end-time and outliers that has >1 lifetime
df_fire = df_fire.dropna(subset=["end_time"])
df_fire = filter_lifetime(df_fire, 365)

In [ ]:
fig, ax = plt.subplots()

ax.plot(df_ystat["year"], df_ystat["count"], color="black", marker="o",
        label="Wildfire Count", ms=4, lw=1, alpha=0.7, linestyle=(0, (1, 0.5)))

ax.set_title("Alberta", fontsize=25)

ax.set_xlabel("Year", fontsize=20)
ax.set_ylabel("Wildfires Count", fontsize=18)
ax.set_xlim(1955, 2025)

ax.tick_params(which="both", direction="in", labelsize=13)
ax.tick_params(which="major", length=6)
ax.tick_params(which="minor", length=4)

# Make the legend
ax.plot([], [], color="orangered", alpha=0.8, lw=1.2, label="Temperature")
ax.legend(loc="lower right", frameon=False, fontsize=10)


# Add the temperature curve
axt = ax.twinx()
axt.plot(years, temp, color="orangered", alpha=0.9, lw=1.2)
axt.set_ylabel(r"Average Temperature ($^\circ C$)", color="orangered",
                fontsize=18, rotation=270, labelpad=20)
axt.tick_params(axis="y", labelcolor="orangered", labelsize=13,
                direction="in")
axt.set_ylim(-1.5, 4)

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df_ystat["year"], df_ystat["avg_size"], color="black", marker="o",
            label="Wildfire Size", ms=4, lw=1, alpha=0.7, linestyle=(0, (1, 0.5)))


ax.set_title("Alberta", fontsize=25)

ax.set_xlabel("Year", fontsize=20)
ax.set_ylabel("Average Burned Area (ha)", fontsize=18)
ax.set_xlim(1980, 2022)
ax.set_yscale("log")

ax.tick_params(which="both", direction="in", labelsize=13)
ax.tick_params(which="major", length=6)
ax.tick_params(which="minor", length=4)

# Make the legend
ax.plot([], [], color="orangered", alpha=0.9, lw=1.2, label="Temperature")
ax.legend(loc="lower right", frameon=False, fontsize=10)


# Add the temperature curve
axt = ax.twinx()
axt.plot(years, temp, color="orangered", alpha=0.8, lw=1.2)
axt.set_ylabel(r"Average Temperature ($^\circ C$)", color="orangered",
                fontsize=18, rotation=270, labelpad=20)
axt.tick_params(axis="y", labelcolor="orangered", labelsize=13,
                direction="in")
axt.set_ylim(-1.5, 4)

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(df_mstat["month"], df_mstat["count"], color="black")

ax.set_title("Alberta")
ax.set_xlabel("Months")
ax.set_ylabel("Number of Wildfires")
ax.set_xlim(0, 13)

axt = ax.twinx()
axt.plot(np.arange(1, 13), temp, color="red")
axt.set_ylabel(r"Temperature ($^\circ C$)", color="red")
axt.tick_params(axis="y", labelcolor="red")


plt.show()

In [ ]:
temp_var = {"temp_st": {"name": r"$\bar{T}_0$", "color": "red"},
            "temp_1d": {"name": r"$\bar{T}_{1}$", "color": "darkorange"},
            "temp_10d": {"name": r"$\bar{T}_{10}$", "color": "darkviolet"},
            "temp_lt": {"name": r"$\bar{T}_{\infty}$", "color": "blue"}
}

bins = np.arange(-20, 50, 1)
centers = (bins[1:] + bins[:-1]) / 2

for v in temp_var:
    hist, _ = np.histogram(df_fire[v].to_numpy(), bins=bins, density=True)
    ax.plot(centers, hist, label=temp_var[v]["name"], 
            color=temp_var[v]["color"])

ax.set_ylabel("Empirical Density", fontsize=20)
ax.set_xlabel("Temperature ($^{\circ}$C)", fontsize=20)
ax.legend(loc="upper right", frameon=False, fontsize=16)
ax.set_title("Alberta", fontsize=25)

plt.show()

In [ ]:
bins = np.arange(-20, 1, 1)
centers = (bins[1:] + bins[:-1]) / 2

ax.hist(df_fire[v].to_numpy(), bins=bins, density=False,
        color="royalblue", edgecolor="white")

ax.set_ylabel("Historgram", fontsize=14)
ax.set_xlabel("Temperature (degree C)", fontsize=14)
ax.set_xticks(np.arange(-20, 1, 2))

plt.show()